In [1]:
import pandas as pd
df_train = pd.read_csv('/content/drive/MyDrive/SMM4H 2024/SMM4H_2024_Task3_Training_1800.csv')
df_dev = pd.read_csv('/content/drive/MyDrive/SMM4H 2024/SMM4H_2024_Task3_Validation_600_release.csv')

In [2]:
df_train = df_train[:1800]
df_train = df_train.iloc[:, :4]
df_train['Classification'] = df_train['Classification'].astype(int)
df_train

,id,keyword,text,Classification
0,3u2w5k,run,"21/m. I want to experience young love, but I'v...",0
1,3xbury,outside,Having issues talking to a girl whom I enjoyed...,0
2,3y743u,"run, swim, climb",Need some advice for free social activities. I...,0
3,43bvs7,walk,I spoke to her today.. A few weeks ago I met ...,0
4,442ap2,outside,"How to get over Social Anxiety?. Hello, The ot...",0
...,...,...,...,...
1795,gqzye9,"pool, beach, pool",Social Anxiety: The Essentials. After looking...,3
1796,env299,"outside , outdoors",Eye contact. Being in public. I can't do it. ...,3
1797,e9bnr1s,Jogging,"If you look for the light, you will find it. I...",3
1798,qrmhbe,"walk, swimming, pool","I feel like my SA is ruining my life. Hi, I h...",3


In [3]:
!pip install datasets -U
!pip install transformers -U
!pip install accelerate -U
!pip install sentencepiece -U
!pip install tqdm -U

In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1800 entries, 0 to 1799
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              1800 non-null   object
 1   keyword         1800 non-null   object
 2   text            1800 non-null   object
 3   Classification  1800 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 56.4+ KB


In [5]:
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
import torch
from torch.utils.data import Dataset
epochs = 5
learning_rate = 2e-5
weight_decay = 1e-2
seed = 117
batch_size = 12
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")
class TextDataset(Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        if self.labels is not None:
            item["labels"] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.encodings["input_ids"])
texts = df_train['text'].tolist()
labels = df_train['Classification'].tolist()  # Ensure your DataFrame has this column for labels
encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)
dataset = TextDataset(encodings, labels)
model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-v3-base", num_labels=4)
training_args = TrainingArguments(
    output_dir='./results',  # Directory for saving model and outputs
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    seed=seed,
    logging_dir='./logs',
    load_best_model_at_end=False,  # Since there's no evaluation, the best model concept does not apply
    save_total_limit=1,  # Limits the saved model checkpoints to the most recent one
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)
trainer.train()
model.save_pretrained("./deberta_task3_model")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:515: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
500,0.749300


In [6]:
torch.cuda.empty_cache()

In [7]:
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from tqdm import tqdm
model_name_or_path = "/content/deberta_task3_model"
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")
model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path)
def get_predictions(text):
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt")
    outputs = model(**inputs)
    probs = outputs.logits.softmax(dim=-1).tolist()[0]
    return probs
for i in tqdm(range(len(df_dev))):
    text = df_dev.loc[i, 'text']
    probs = get_predictions(text)
    df_dev.loc[i, 'prob_label_0'] = probs[0]
    df_dev.loc[i, 'prob_label_1'] = probs[1]
    df_dev.loc[i, 'prob_label_2'] = probs[2]
    df_dev.loc[i, 'prob_label_3'] = probs[3]
df_dev

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:515: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
100%|██████████| 600/600 [03:36<00:00,  2.77it/s]


,id,keyword,text,Label,prob_label_0,prob_label_1,prob_label_2,prob_label_3
0,3p7iya,bay,Advice and tips on how to improve my public s...,NaN,0.985697,0.002299,0.011149,0.000855
1,3qm3au,run,"Getting into a new social life (moving out, n...",NaN,0.983601,0.002242,0.013255,0.000903
2,3sy0nb,"river, river","Do any of you guys have moments of clarity, wh...",NaN,0.951149,0.003396,0.043057,0.002399
3,3y7rva,running,Self worth and anxiety. Hi guys. 25 (M) I am n...,NaN,0.012454,0.230407,0.220188,0.536951
4,43xj5z,"coast, coast",I feel like life's going to pass me by.. Eith...,NaN,0.990868,0.002127,0.006352,0.000652
...,...,...,...,...,...,...,...,...
595,rrb9y6,outside,"I just moved out alone, and I'm terrified of e...",NaN,0.242122,0.011273,0.729769,0.016835
596,rwa3vy,run,Need advice on how to help my best friend with...,NaN,0.974873,0.003191,0.020164,0.001772
597,s11dzj,run,Is it just me or is it normal for people to be...,NaN,0.988204,0.002601,0.008209,0.000986
598,t0wlip,stream,It looks like I am giving up because of my so...,NaN,0.971598,0.002723,0.024203,0.001476


In [8]:
df_dev.to_csv('/content/DeBERTa-001-preds.csv')